## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-05-49-06 +0000,nypost,Moment feds descend on suspected Charlie Kirk ...,https://nypost.com/2025/09/13/us-news/utah-fed...
1,2025-09-13-05-48-36 +0000,missionlocal,Venezuelans offered 24 hours to re-register fo...,https://missionlocal.org/2025/09/venezuelans-o...
2,2025-09-13-05-43-47 +0000,bbc,Prince Harry makes surprise Ukraine visit to s...,https://www.bbc.com/news/articles/c3dr859vxxjo...
3,2025-09-13-05-29-06 +0000,bbc,"Rising cost of school uniform is scary, says mum",https://www.bbc.com/news/articles/c179ev9ge5qo...
4,2025-09-13-05-07-17 +0000,nypost,Fired MSNBC analyst Matthew Dowd defends twist...,https://nypost.com/2025/09/13/us-news/fired-ms...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,kirk,54
4,charlie,49
50,trump,24
109,shooting,16
110,suspect,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
282,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,185
23,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,153
109,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,153
13,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,153
53,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,151


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
282,185,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
26,59,2025-09-13-02-07-47 +0000,nypost,Missouri passes Trump-backed redistricting pla...,https://nypost.com/2025/09/12/us-news/missouri...
291,52,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
23,50,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...
299,42,2025-09-12-08-00-00 +0000,cbc,"Israel's attack on Qatar was an escalation, bu...",https://www.cbc.ca/news/world/israel-qatar-inv...
45,40,2025-09-13-00-40-07 +0000,nypost,Charlie Kirk’s wife Erika delivers emotional f...,https://nypost.com/2025/09/12/us-news/charlie-...
62,35,2025-09-12-23-35-31 +0000,nypost,Tyler Robinson may have revealed details about...,https://nypost.com/2025/09/12/us-news/tyler-ro...
194,33,2025-09-12-15-18-57 +0000,startribune,Another shocking act of political violence lea...,https://www.startribune.com/charlie-kirk-minne...
6,33,2025-09-13-04-48-02 +0000,bbc,New Nato mission to bolster eastern flank afte...,https://www.bbc.com/news/articles/c0q719qg1wko...
258,32,2025-09-12-10-56-14 +0000,nypost,Fashion designer Giorgio Armani’s will instruc...,https://nypost.com/2025/09/12/business/fashion...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
